ジェンダーを軸にして、クロス集計する

In [1]:
import altair as alt
import pandas as pd
import scipy as sp
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

Altair 5.0.1
Pandas 2.0.3
SciPy 1.11.1
Titanite 0.2.0


データを読み込む

In [2]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

2023-07-22 19:08:22.021 | INFO     | titanite.preprocess:categorical_data:123 - Categorize
2023-07-22 19:08:22.037 | INFO     | titanite.preprocess:binned_data:229 - Binned


- クロス集計は``pd.crosstab(index, columns)``を使う
  - ``values=None``, ``aggfunc=None``: セットで指定する。``values``をどのように``aggfunc``するか。
  - ``margins=False``, ``margins_name="All"``
  - ``dropna=True`` : すべての値がNaNのカラムは除外する
  - ``normalize=False``
- カイ二乗検定は``scipy.stats.chi2_contigency(observed)``を使う
  - ``correction=True``: 自由度が1のとき、Yates'の補正をする。集計数が少ないときにp-valueを大きくする補正。
  - ``lambda_=None``: 検定方法（分布？モデル？）を変更できる。デフォルトはピアソン。

In [3]:
x = "q02"

headers = [header for header in data.columns if header not in d.ignore_from_crosstab]
headers.remove(x)

cross_tabs = {}
heatmaps = {}
chi2_tests = []

for y in headers:
    name = f"{x}-{y}"
    cross_tab, chi2_test, heatmap = ti.core.crosstab(data, y, x)
    cross_tabs.update({name: cross_tab})
    heatmaps.update({name: heatmap})
    _data = [name, chi2_test.statistic, chi2_test.pvalue, chi2_test.dof]
    chi2_tests.append(_data)

chi2_data = pd.DataFrame(chi2_tests, columns=["questions", "statistic", "p-value", "dof"])

In [4]:
chi2_data

,questions,statistic,p-value,dof
0,q02-q01,29.775320,0.579596,32
1,q02-q03,71.488371,0.243178,64
2,q02-q04,77.894940,0.192958,68
3,q02-q05,21.097674,0.632957,24
4,q02-q06,73.110661,0.000047,32
5,q02-q07,4.887485,0.769531,8
6,q02-q08,63.665246,0.000002,20
7,q02-q09,9.482135,0.303268,8
8,q02-q11,12.849407,0.117137,8
9,q02-q12_genderbalance,16.524754,0.683582,20


In [5]:
mark = alt.Chart(chi2_data).mark_point().encode(
    alt.X("p-value"),
    alt.Y("questions"),
    tooltip=["questions", "p-value", "dof", "statistic"]
)

rule = (
    alt.Chart()
    .mark_rule(color="orange")
    .encode(x=alt.datum(0.05))
)

(mark + rule).properties(
    width=500,
    height=500,
).interactive()

alt.LayerChart(...)